In [5]:
import mysql.connector
from mysql.connector import Error
import time

def connect_to_mysql():
    print("=== KẾT NỐI MYSQL DOCKER ===\n")
    
    # Thử kết nối không chỉ định database trước
    config_no_db = {
        'host': '127.0.0.1',
        'port': 3307,
        'user': 'root',
        'password': 'uneti',  # ĐÚNG PASSWORD
        'connection_timeout': 10
    }
    
    config_with_db = {
        'host': '127.0.0.1',
        'port': 3307,
        'user': 'root',
        'password': 'uneti',
        'database': 'data_science_db',  # CHỈ ĐỊNH DATABASE
        'connection_timeout': 10
    }
    
    try:
        print("1. Thử kết nối không chỉ định database...")
        conn = mysql.connector.connect(**config_no_db)
        
        if conn.is_connected():
            print("✅ Kết nối thành công đến MySQL server")
            
            cursor = conn.cursor()
            
            # Kiểm tra phiên bản
            cursor.execute("SELECT VERSION()")
            version = cursor.fetchone()
            print(f"• MySQL Version: {version[0]}")
            
            # Kiểm tra các database
            cursor.execute("SHOW DATABASES")
            databases = cursor.fetchall()
            print(f"• Tổng số database: {len(databases)}")
            
            # Liệt kê tất cả database
            db_list = [db[0] for db in databases]
            print(f"• Databases: {', '.join(db_list)}")
            
            # Kiểm tra data_science_db có tồn tại không
            if 'data_science_db' in db_list:
                print("✅ Database 'data_science_db' tồn tại")
                
                # Đóng kết nối cũ
                cursor.close()
                conn.close()
                
                # Kết nối với database cụ thể
                print("\n2. Kết nối đến database 'data_science_db'...")
                conn2 = mysql.connector.connect(**config_with_db)
                
                if conn2.is_connected():
                    print("✅ Đã kết nối đến data_science_db")
                    
                    cursor2 = conn2.cursor()
                    cursor2.execute("SELECT DATABASE()")
                    current_db = cursor2.fetchone()
                    print(f"• Database hiện tại: {current_db[0]}")
                    
                    # Tạo bảng demo nếu chưa có
                    cursor2.execute("""
                        CREATE TABLE IF NOT EXISTS demo_table (
                            id INT AUTO_INCREMENT PRIMARY KEY,
                            name VARCHAR(100),
                            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
                        )
                    """)
                    print("• Đã tạo/kiểm tra bảng demo_table")
                    
                    cursor2.execute("INSERT INTO demo_table (name) VALUES ('Test Data')")
                    conn2.commit()
                    
                    # Đếm số bảng
                    cursor2.execute("SHOW TABLES")
                    tables = cursor2.fetchall()
                    print(f"• Số bảng trong data_science_db: {len(tables)}")
                    print(f"• Tables: {', '.join([t[0] for t in tables])}")
                    
                    cursor2.close()
                    conn2.close()
                    print("\n✅ KẾT NỐI HOÀN TẤT!")
                    
            else:
                print("❌ Database 'data_science_db' KHÔNG tồn tại")
                print("Tạo database mới...")
                
                cursor.execute("CREATE DATABASE data_science_db")
                print("✅ Đã tạo database 'data_science_db'")
                
                cursor.close()
                conn.close()
                
            return True
            
    except Error as e:
        print(f"❌ Lỗi kết nối: {e}")
        
        # Gợi ý giải pháp
        if "Unknown database" in str(e):
            print("\n⚠ GỢI Ý SỬA LỖI:")
            print("1. Chạy lệnh tạo database: CREATE DATABASE data_science_db;")
            print("2. Hoặc xóa và chạy lại container với đúng tham số")
            
        elif "Access denied" in str(e):
            print("\n⚠ GỢI Ý SỬA LỖI:")
            print("1. Kiểm tra password (phải là 'uneti')")
            print("2. Kiểm tra username (phải là 'root')")
            
        return False

if __name__ == "__main__":
    # Thử 3 lần
    for i in range(3):
        print(f"\n--- Lần thử {i+1} ---")
        if connect_to_mysql():
            break
        if i < 2:
            print(f"\nChờ 5 giây và thử lại...")
            time.sleep(5)



--- Lần thử 1 ---
=== KẾT NỐI MYSQL DOCKER ===

1. Thử kết nối không chỉ định database...
✅ Kết nối thành công đến MySQL server
• MySQL Version: 9.5.0
• Tổng số database: 4
• Databases: information_schema, mysql, performance_schema, sys
❌ Database 'data_science_db' KHÔNG tồn tại
Tạo database mới...
✅ Đã tạo database 'data_science_db'
